In [8]:
import random
import math
import gc
import os

from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from imutils.paths import list_files
import torchvision.utils as vutils
import torchvision.io
import numpy as np
import torch
import cv2

from utils import calculate_psnr, calculate_ssim
from utils import transforms, VideoDataset
from vivq import VIVQ

In [9]:
gc.collect()
torch.cuda.empty_cache()

In [10]:
SEP        = os.path.sep
ROOT_PATH  = SEP.join(os.getcwd().split(SEP)[:-3])

DATA_PATH  = f'{ROOT_PATH}/DataSets'
ROOT_PATH, DATA_PATH

('/home/projects', '/home/projects/DataSets')

In [11]:
def load_video(path, clip_len = 10, skip_frames = 3):

    video_transform = torchvision.transforms.Compose([
                        torchvision.transforms.Resize(128),
                        torchvision.transforms.CenterCrop(128)
                    ])

    video, _, _ = torchvision.io.read_video(path)
    video       = video.permute(0, 3, 1, 2) / 255.
    max_seek    = video.shape[0] - (clip_len * skip_frames)

    start       = math.floor(random.uniform(0., max_seek))
    # video       = video[:30]

    if video_transform:
        video = video_transform(video)

    image, video = video[0], video[1: ]
    return image.unsqueeze(0), video.unsqueeze(0)

In [12]:
sample_vid = '/home/projects/DataSets/kinetic400/tossing_coin/TMzJ-y3Ne5k.mp4'
# sample_vid = '/home/projects/DataSets/Moments_in_Time_Raw/validation/drenching/yt-jU0lcu7Bjrk_871.mp4'
# sample_vid = '/home/projects/DataSets/webvid/validation/00834012.mp4'

In [13]:
name        = 'vivq_2'
device      = 'cuda'
num_frames  = 50
skip_frames = 5
hidden_C    = 512

In [14]:
for epoch in [270_000]:

    print(f'[{epoch}]')
    MODEL_PATH = f'{ROOT_PATH}/video_generation/phenaki/phenaki/models/vivq_8192_drop_video_k400/model_{epoch}.pt'
    model      = VIVQ(c_hidden = hidden_C, codebook_size = 8192).to(device)
    state_dict = torch.load(MODEL_PATH)
    
    model.load_state_dict(state_dict)
    model.eval().requires_grad_(False)
    
    print('모델 로딩 완')
    
    image, video = load_video(sample_vid, clip_len = num_frames, skip_frames = skip_frames)
    image, video = image.to(device), video.to(device)

    recon, _    = model(image, video)
    folder_name = sample_vid.split(SEP)[-2]
    file_name   = os.path.splitext(sample_vid.split(SEP)[-1])[0]
    
    os.makedirs(f'results/valid/{folder_name}', exist_ok = True)
    
    orig  = torch.cat([image.unsqueeze(1), video], dim = 1)
    orig  = orig[0]
    recon = recon[0]

    save_path = f'results/valid/{folder_name}/{file_name}_{str(epoch).zfill(6)}_{num_frames}'
    comp = vutils.make_grid(torch.cat([orig, recon]), nrow = len(orig)).detach().cpu()
    vutils.save_image(comp, f'{save_path}.jpg')
    
    origs_  = orig.detach().cpu()
    recons_ = recon.detach().cpu()

    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    output = cv2.VideoWriter(f'{save_path}.mp4', fourcc, 30.0, (128, 128))
    PSNR_, SSIM_ = [], []
    for ori, rec in zip(origs_, recons_):


        rec_   = np.transpose(rec.detach().cpu().numpy(), (1, 2, 0)) * 255
        rec_   = cv2.cvtColor(rec_, cv2.COLOR_BGR2RGB)
        output.write(rec_.astype(np.uint8))

        
        ori = np.transpose(ori.detach().cpu().numpy(), (1, 2, 0))
        rec = np.transpose(rec.detach().cpu().numpy(), (1, 2, 0))
    
        PSNR_.append(calculate_psnr(ori,rec))
        SSIM_.append(calculate_ssim(ori, rec))

    output.release()
    
    print(f'PSNR : {np.mean(PSNR_):.3f}, SSIM : {np.mean(SSIM_):.3f}')
    plt.imshow(comp.permute(1, 2, 0))
    plt.axis(False)
    plt.show()
    print('\n\n')

[270000]
모델 로딩 완
PSNR : 23.552, SSIM : 0.427
